# 1. Imports

In [ ]:
import os
import json
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import sys
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CocoDetection
from torch.cuda.amp import autocast, GradScaler
from datetime import datetime
#import onnx
#import onnxruntime as ort
from PIL import Image
import numpy as np


base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(base_dir)

sys.path.append(base_dir)

%matplotlib inline

# 2. Funcoes

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Definir o dispositivo (GPU ou CPU)

class CustomTransform: # Transformação das imagens
    def __call__(self, img, target):
        if isinstance(img, torch.Tensor):
            return img.to(device), target  # Verifica se a imagem é tensor, caso seja já envia para o dispositivo

        img = transforms.ToTensor()(img)  # Caso não seja Tensor, transforma em tensor
        return img.to(device), target  # Envia a imagem para o dispositivo
    
class CustomDataset(CocoDetection): # Configuração do dataset personalizado
    def __init__(self, root, annotation, transforms=None):
        super().__init__(root, annotation)
        self.transforms = transforms

    def __getitem__(self, idx):
        img, target = super().__getitem__(idx)

        if self.transforms:
            img, target = self.transforms(img, target)  # Aplica a transformação na imagem e no target

        return img, target

def load_settings():
    """Função para ler as configurações de um arquivo .txt e atribuir os valores diretamente às variáveis."""
    # Valores padrão
    defaults = {
        'perc_top': 0.4,
        'perc_bottom': 0.8,
        'min_score': 0.5,
        'limit_center': 8,
        'save_dir': 'data\\outputs\\capturas',
        'square_size': 640,
        'grid_x': 0,
        'grid_y': 0,
        'crop_image': 1  # 1 = Sim
    }

    return (
        defaults['perc_top'],
        defaults['perc_bottom'],
        defaults['min_score'],
        defaults['limit_center'],
        defaults['save_dir'],
        defaults['square_size'],
        defaults['grid_x'],
        defaults['grid_y'],
        defaults['crop_image'],
    )

def select_image(images_dir, image_choice):
    base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

    # Diretório onde estão as imagens e as anotações
    annotations_path = fr'{images_dir}/_annotations.coco.json'  

    # Carregar anotações COCO
    with open(os.path.join(base_dir, annotations_path)) as f:
        coco_data = json.load(f)

    # Criar um dicionário para mapear IDs de imagens para anotações
    annotations_dict = {}
    for annotation in coco_data['annotations']:
        image_id = annotation['image_id']
        if image_id not in annotations_dict:
            annotations_dict[image_id] = []
        annotations_dict[image_id].append(annotation)
    images_dict = {image['id']: image['file_name'] for image in coco_data['images']}

    if 0 <= image_choice < len(coco_data['images']):
        image_info = coco_data['images'][image_choice]
        image_file = image_info['file_name']
        image_path = os.path.join(base_dir, images_dir, image_file)
        image_id = image_info['id']
        
        # Obter anotações para a imagem selecionada
        annotations = annotations_dict.get(image_id, [])
        
        return image_path, annotations, image_file
    
def visualize_coco_all_annotations(image_path, annotations, image_name):
    # Carregar a imagem original
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converter para RGB para exibição com matplotlib
    
    # Criar uma cópia para desenhar anotações
    annotated_image = image_rgb.copy()
    
    # Exibir informações da imagem
    height, width, channels = image.shape
    print(f"Exibindo imagem: {image_name}")
    print(f"Dimensões da imagem: {width}x{height} (largura x altura, {channels} canais)")

    # Desenhar as bounding boxes na imagem anotada
    for annotation in annotations:
        x, y, w, h = annotation['bbox']
        cv2.rectangle(annotated_image, (int(x), int(y)), (int(x + w), int(y + h)), color=(255, 0, 0), thickness=2)
        print(f"Coordenadas da bounding box: x={x}, y={y}, width={w}, height={h}")
    
    # Criar uma figura com 2 subplots: imagem original e anotada
    plt.figure(figsize=(16, 8))

    # Subplot 1: imagem original
    plt.subplot(1, 2, 1)
    plt.imshow(image_rgb)
    plt.title("Imagem Original")
    plt.axis('off')

    # Subplot 2: imagem com anotações
    plt.subplot(1, 2, 2)
    plt.imshow(annotated_image)
    plt.title(f"Imagem com anotações: {len(annotations)} detecções")
    plt.axis('off')

    plt.show()

def create_model(weights, num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=weights)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

    return model

def load_pretrained_model(device, model, pretrained_model_path):
    if pretrained_model_path and os.path.exists(pretrained_model_path):
        model.load_state_dict(torch.load(pretrained_model_path, map_location=device))
        print(f"Modelo carregado de: {pretrained_model_path}")

    else:
        print("Treinando modelo do zero...")

    model.to(device)
    return model

def load_model_eval(device, model, pretrained_model_path):  # Colocar o modelo em modo de avaliação
    model.load_state_dict(torch.load(pretrained_model_path, map_location=device))
    model.to(device)
    model.eval() 
    
    return model

def dataset_config(images_dir, annotations_path, data_loader_batch_size):
    dataset = CustomDataset(
        root=os.path.join(base_dir, images_dir),
        annotation=os.path.join(base_dir, annotations_path),
        transforms=CustomTransform()  # Passa a transformação customizada
    )

    # Configuração do DataLoader
    data_loader = DataLoader(dataset, batch_size=data_loader_batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

    return dataset, data_loader

def load_image_tensor(image_path):
    image = Image.open(image_path).convert("RGB")  # Carrega a imagem e converte para RGB
    transform = transforms.ToTensor()  # Define a transformação para ToTensor
    image_tensor = transform(image).unsqueeze(0)  # Converte a imagem para tensor e adiciona uma dimensão de batch
    
    return image_tensor

def rules_detection(frame, detections_sorted, perc_top, perc_bottom, min_score, limit_center):
    height, width = frame.shape[:2]

    # Define as posições das linhas
    line_limit_top = int(height * perc_top)    # Só conta quando a mediana entrar nesse range
    line_limit_bottom = int(height * perc_bottom)   # Só conta quando a mediana entrar nesse range
    dif_limit = line_limit_bottom - line_limit_top

    # Linha de limite superior
    cv2.line(
        frame, 
        (0, line_limit_top), 
        (frame.shape[1], line_limit_top), 
        (0, 255, 0),  # Verde
        2,
    )

    # Linha de limite inferior
    cv2.line(
        frame, 
        (0, line_limit_bottom), 
        (frame.shape[1], line_limit_bottom), 
        (0, 255, 0),  # Verde
        2,
    )

    line_top_detect = None  # É calculada a mediana das detecções, e só são contados biscoitos que estão naquela mediana + - um valor de range
    line_bottom_detect = None   # É calculada a mediana das detecções, e só são contados biscoitos que estão naquela mediana + - um valor de range

    # Lista para armazenar centros de detecções
    centers = []
    all_centers_y = ([])  # Lista para armazenar as coordenadas y dos centros detectados

    total_detections = 0  # Contador total de detecções

    ### CALCULO DA MEDIANA
    for idx, detection in enumerate(detections_sorted):
        score = detection[1]   # Pontuação de confiança
        x_min, y_min, x_max, y_max = detection[0]  # Coordenadas da caixa - y cresce de cima para baixo

        # Verificar se a pontuação é maior que o limite e se está entre as linhas de contagem
        if (score > min_score and y_max > line_limit_top and y_min < line_limit_bottom):
            center_y = (y_min + y_max) // 2   # Calcular o centro da caixa de detecção
            all_centers_y.append(center_y)  # Adiciona y à lista de centros

    if all_centers_y:
        median_y = int(np.median(all_centers_y))  # Obtém a mediana
        line_bottom_detect = int(median_y + int(dif_limit / 2))
        line_top_detect = int(median_y - int(dif_limit / 2))
        
        cv2.line(
            frame, 
            (640, median_y), 
            (640 + 640, median_y), 
            (255, 0, 0), 
            2,
        )  # Desenhar a linha horizontal na moda

    ### MARCACAO
    if ( all_centers_y and line_limit_bottom > median_y > line_limit_top):   # Se tiver pelo menos uma marcação dentro dos limites e a mediana for dentro dos limites
        for idx, detection in enumerate(detections_sorted):
            score = detection[1]   # Pontuação de confiança
            x_min, y_min, x_max, y_max = detection[0]  # Coordenadas da caixa - y cresce de cima para baixo

            center_x = int((x_min + x_max) // 2)   # Calcular o centro da caixa de detecção
            center_y = int((y_min + y_max) // 2)

            test_score = score > min_score   # Verifica score da deteccao
            test_center = not any(np.linalg.norm(np.array([center_x, center_y]) - np.array(center))< limit_center for center in centers)   # Verifica se está próximo de algum centro
            test_center_x = not any(abs(center_x - center[0]) < limit_center for center in centers)   # Verifica se está próximo de algum x dos centros
            test_median = (y_max > line_top_detect and y_min < line_bottom_detect)   # Verifica se está na mediana +- range

            if test_score and test_center and test_median:
                total_detections += 1
                # print(f"Deteccao {str(total_detections)} ({idx + 1}): {str(score)} - OK (Score: {test_score} / Center: {test_center} / Center X: {test_center_x} / Median: {test_median})")

                centers.append((center_x, center_y))  # Adiciona o centro à lista

                cv2.circle(
                    frame, 
                    (center_x, center_y), 
                    limit_center, # Caso nao queira que seja igual o limit_center, tirar o comentado abaixo e colocar aqui um numero fixo
                    (0, 0, 255), 
                    -1,
                )   # Desenhar uma bolinha (círculo) no centro

                cv2.circle(
                    frame, 
                    (center_x, center_y), 
                    limit_center, 
                    (255, 0, 0), 
                    1,
                )  # Círculo vermelho de limite

                # cv2.putText(
                #     frame,
                #     str(total_detections),
                #     (center_x - 6, center_y + 3),
                #     cv2.FONT_HERSHEY_SIMPLEX,0.35,(255, 255, 255),
                #     1,
                # )   # Colocar o número da marcação dentro da bolinha

    cv2.line(
        frame, 
        (640, line_top_detect), 
        (640 + 640, line_top_detect), 
        (255, 0, 0), 
        2,

    )
    cv2.line(
        frame,
        (640,
         line_bottom_detect),
         (640 + 640, line_bottom_detect),
         (255, 0, 0),
         2,
    )

    return frame, total_detections

def no_rules_detection(frame, detections_sorted, perc_top, perc_bottom, min_score, limit_center):
    total_detections = 0  # Contador total de detecções

    for idx, detection in enumerate(detections_sorted):
        score = detection[1]
        x_min, y_min, x_max, y_max = detection[0]

        center_x = int((x_min + x_max) // 2)
        center_y = int((y_min + y_max) // 2)

        total_detections += 1

        # Marca o centro com um círculo vermelho
        cv2.circle(
            frame,
            (center_x, center_y),
            7,
            (0, 0, 255),
            -1,
        )

        # Círculo azul indicando o raio que antes era o limit_center
        cv2.circle(
            frame,
            (center_x, center_y),
            limit_center,
            (255, 0, 0),
            1,
        )

        # Número da detecção
        cv2.putText(
            frame,
            str(total_detections),
            (center_x - 6, center_y + 3),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.35,
            (255, 255, 255),
            1,
        )

    # Área superior de fundo do texto
    cv2.rectangle(
        frame,
        (0, 0),
        (frame.shape[1], 40),
        (80, 43, 30),
        -1,
    )

    # Texto de total
    text = f'Total de Biscoitos: {total_detections}'
    cv2.putText(
        frame,
        text,
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (255, 255, 255),
        2,
    )

    return frame, total_detections

def visualize_predictions_image(print_images, device, model, image_tensor, coco_annotations=None, threshold=0.5, limit_center = 6, perc_top = 0.35, perc_bottom = 0.6):
    # Tratamento inicial das imagens
    image_np = image_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
    image_detected = (image_np * 255).astype(np.uint8).copy()  # OpenCV usa 0-255
    image_annotated = image_np.copy()

    frame_bgr = cv2.cvtColor(image_detected, cv2.COLOR_RGB2BGR)

    # Regras nas anotacoes
    height, width = frame_bgr.shape[:2]
    line_limit_top = int(height * perc_top)    # Só conta quando a mediana entrar nesse range
    line_limit_bottom = int(height * perc_bottom)   # Só conta quando a mediana entrar nesse range

    coco_annotations_rules = []
    if coco_annotations:
        for ann in coco_annotations:
            x, y, w, h = ann['bbox']
            y_min = y
            y_max = y + h

            # Filtrar somente as anotações dentro da faixa de interesse
            if y_max > line_limit_top and y_min < line_limit_bottom:
                coco_annotations_rules.append(ann)

    # Regras na deteccao
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        predictions = model(image_tensor)

    prediction = predictions[0]

    boxes_pred = prediction['boxes'].cpu().detach().numpy()
    scores = prediction['scores'].cpu().detach().numpy()

    # Preparar detecções válidas no formato necessário
    detections_sorted = [
        (box, score) for box, score in zip(boxes_pred, scores) if score > threshold
    ]

    # Aplicar regras na imagem de detecção (OpenCV)
    frame_bgr, total_detections = rules_detection(
        frame=frame_bgr,
        detections_sorted=detections_sorted,
        perc_top=perc_top,
        perc_bottom=perc_bottom,
        min_score=threshold,
        limit_center=limit_center,
    )

    print(f"Anotacoes: {len(coco_annotations_rules)} / Deteccoes: {total_detections}")
    # Criar subplots
    if print_images == True:
        fig, ax = plt.subplots(1, 3, figsize=(24, 8))

        # Imprimir Imagem Original
        ax[0].imshow(image_np)
        ax[0].axis('off')
        ax[0].set_title('Imagem Original', fontsize=16)

        # Imprimir Imagem com Anotações
        ax[1].imshow(image_annotated)
        
        # Linha de limite superior
        ax[1].axhline(y=line_limit_top, color='green', linewidth=2)

        # Linha de limite inferior
        ax[1].axhline(y=line_limit_bottom, color='green', linewidth=2)

        ax[1].set_title(f'Imagem com Anotações: {len(coco_annotations_rules)}', fontsize=16)

        ax[1].axis('off')

        if coco_annotations_rules:
            for ann in coco_annotations_rules:
                x, y, w, h = ann['bbox']
                y_min = y
                y_max = y + h

                # Filtrar somente as anotações dentro da faixa de interesse
                if y_max > line_limit_top and y_min < line_limit_bottom:
                    rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='blue', facecolor='none')
                    ax[1].add_patch(rect)

        # Imprimir Imagem com Detecções
        image_detected = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        ax[2].imshow(image_detected)
        ax[2].axis('off')
        ax[2].set_title(f'Imagem com Detecções Regradas: {total_detections}', fontsize=16)


        plt.tight_layout()
        plt.show()
    
    result = len(coco_annotations_rules) == total_detections
    return result


In [ ]:
###################################### Modelo ######################################
# Iniciar
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Definir o dispositivo (GPU ou CPU)

# Inputs
save_dir = os.path.join(base_dir, 'data', 'inputs', 'ia_models', 'FRCNN Resnet50')
models_path = os.path.join(base_dir, 'data', 'inputs', 'ia_models', 'FRCNN Resnet50')
pretrained_model_path = os.path.join(models_path, 'best_faster_rcnn_model_20250520_zoom.pth')  # Altere para o caminho do modelo salvo (ex: 'caminho/para/modelo.pth') ou mantenha None para treinar do zero
weights='FasterRCNN_ResNet50_FPN_Weights.COCO_V1'
num_classes = 2  # Inclua o número de classes (background + classes)
num_epochs = 10000
data_loader_batch_size = 1

model = create_model(weights, num_classes)
model = load_pretrained_model(device, model, pretrained_model_path)
model = load_model_eval(device, model, pretrained_model_path)

# 3. Individual

In [ ]:
images_dir = r'data/inputs/train_images/COCO_2025.05_group/valid'  
annotations_path = r'data/inputs/train_images/COCO_2025.05_group/valid/_annotations.coco.json'  
image_choice = 20  # Escolher a imagem (ajuste conforme necessário)

image_path, annotations, image_file = select_image(images_dir, image_choice)
print(image_path)

print(f"Modelo no modo de avaliação no dispositivo: {next(model.parameters()).device}") # Verificar se o modelo está no dispositivo correto (GPU ou CPU)

image_tensor = load_image_tensor(image_path)
result = visualize_predictions_image(False, device, model, image_tensor, annotations, threshold=0.5, perc_top=0.1, perc_bottom=0.5)
print(result)
######################################################################################


# 4 Check Geral

In [ ]:
def check_geral(threshold, limit_center, print_images = False):
    images_dir = r'data/inputs/train_images/COCO_2025.05_group/valid'  
    annotations_path = r'data/inputs/train_images/COCO_2025.05_group/valid/_annotations.coco.json'  

    total_images = 0
    total_true = 0

    ############################# 000 #############################
    image_choice = 0  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.1, perc_bottom=0.5)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 001 #############################
    image_choice = 1  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.4, perc_bottom=0.7)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 003 #############################
    image_choice = 3  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.5)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 004 #############################
    image_choice = 4  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.5, perc_bottom=0.9)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 005 #############################
    image_choice = 5  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.5, perc_bottom=0.9)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 006 #############################
    image_choice = 6  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.45, perc_bottom=0.9)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 007 #############################
    image_choice = 7  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.45, perc_bottom=0.9)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 008 #############################
    image_choice = 8  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.45, perc_bottom=0.9)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 009 #############################
    image_choice = 9  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.5)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 010 #############################
    image_choice = 10  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.5, perc_bottom=0.8)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 011 #############################
    image_choice = 11  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.6)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 012 #############################
    image_choice = 12  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.6)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 013 #############################
    image_choice = 13  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.5, perc_bottom=0.9)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 014 #############################
    image_choice = 14  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.6)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 015 #############################
    image_choice = 15  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.6)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 016 #############################
    image_choice = 16  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.15, perc_bottom=0.5)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 017 #############################
    image_choice = 17  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.5, perc_bottom=0.85)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 018 #############################
    image_choice = 18  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.1, perc_bottom=0.5)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 019 #############################
    image_choice = 19  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.3, perc_bottom=0.55)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 020 #############################
    image_choice = 20  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.1, perc_bottom=0.4)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 021 #############################
    image_choice = 21  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.35, perc_bottom=0.6)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 022 #############################
    image_choice = 22  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.55)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 023 #############################
    image_choice = 23  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.55)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 024 #############################
    image_choice = 24  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.55)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 025 #############################
    image_choice = 25  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0, perc_bottom=0.3)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 026 #############################
    image_choice = 26  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.4, perc_bottom=0.75)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 027 #############################
    image_choice = 27  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.4, perc_bottom=0.75)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 028 #############################
    image_choice = 28  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.1, perc_bottom=0.45)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 029 #############################
    image_choice = 29  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.45, perc_bottom=0.8)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 030 #############################
    image_choice = 30  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.45, perc_bottom=0.8)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 031 #############################
    image_choice = 31  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.6, perc_bottom=0.9)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 032 #############################
    image_choice = 32  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.6, perc_bottom=0.95)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 033 #############################
    image_choice = 33  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.4, perc_bottom=0.7)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 034 #############################
    image_choice = 34  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.35, perc_bottom=0.65)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 035 #############################
    image_choice = 35  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.4, perc_bottom=0.75)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 036 #############################
    image_choice = 36  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.5, perc_bottom=0.8)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 037 #############################
    image_choice = 37  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.5)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 038 #############################
    image_choice = 38  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.5)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 039 #############################
    image_choice = 39  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.3, perc_bottom=0.6)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 040 #############################
    image_choice = 40  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.4, perc_bottom=0.7)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 041 #############################
    image_choice = 41  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.35, perc_bottom=0.8)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 042 #############################
    image_choice = 42  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.25, perc_bottom=0.6)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 043 #############################
    image_choice = 43  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.4, perc_bottom=0.80)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 044 #############################
    image_choice = 44  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.2, perc_bottom=0.5)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 045 #############################
    image_choice = 45  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.05, perc_bottom=0.4)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 046 #############################
    image_choice = 46  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.05, perc_bottom=0.4)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 047 #############################
    image_choice = 47  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.4, perc_bottom=0.7)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 048 #############################
    image_choice = 48  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.4, perc_bottom=0.7)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 049 #############################
    image_choice = 49  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.3, perc_bottom=0.6)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 050 #############################
    image_choice = 50  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.25, perc_bottom=0.6)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 051 #############################
    image_choice = 51  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.1, perc_bottom=0.4)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 052 #############################
    image_choice = 52  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.1, perc_bottom=0.4)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 053 #############################
    image_choice = 53  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.25, perc_bottom=0.6)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 054 #############################
    image_choice = 54  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.6, perc_bottom=0.9)
    total_images += 1
    if result == True:
        total_true += 1

    ############################# 054 #############################
    image_choice = 54  # Escolher a imagem (ajuste conforme necessário)
    image_path, annotations, image_file = select_image(images_dir, image_choice)
    print(image_path)
    image_tensor = load_image_tensor(image_path)
    result = visualize_predictions_image(print_images, device, model, image_tensor, annotations, threshold=threshold, limit_center=limit_center, perc_top=0.6, perc_bottom=0.9)
    total_images += 1
    if result == True:
        total_true += 1


    print('--------------------------------------------------------------------------------------')
    print(f' Resultado Final: {total_true/total_images:.2f}%')

    return total_true/total_images

In [ ]:
threshold = 0.5
limit_center = 6
result = check_geral(threshold=threshold, limit_center=limit_center, print_images=True)

In [ ]:
threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
limit_center_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 19]

print_images = False

best_result = 0.0
for threshold in threshold_list:
    for limit_center in limit_center_list:
        result = check_geral(threshold=threshold, limit_center=limit_center)

        if result > best_result:
            best_result = result
            best_threhold = threshold
            best_limit_center = limit_center

In [ ]:
print(best_result)
print(best_threhold)
print(best_limit_center)